## 1. Import các thư viện cần thiết

In [1]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.MZxilADFHH/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.hngoiS4Bkl/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.GbXUEAUj4M/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://deb.debian.org/debian bust

In [2]:
import pandas as pd
import os
import requests
import time
import random

from io import BytesIO
from PIL import Image
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## 2. Crawl

In [3]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.headless = True
driver = webdriver.Chrome('chromedriver',options=chrome_options)
driver.implicitly_wait(2)

root_dir = './space_image_captioning_dataset'
images_dir = os.path.join(
    root_dir, 
    './space_images'
)
captions_path = os.path.join(
    root_dir, 
    './space_captions.txt'
)

os.makedirs(root_dir, exist_ok=True)
os.makedirs(images_dir, exist_ok=True)
caption_file = open(captions_path, 'w', encoding='utf8')

img_idx = 1
try:
    for page_idx in tqdm(range(1, 100)):
        main_url = f'https://kienthuckhoahoc.org/kh-vu-tru/page{page_idx}' 
        driver.get(main_url)

        news_lst_xpath = '//a[@class="title"]'
        news_tags = driver.find_elements(
            By.XPATH, 
            news_lst_xpath
        )

        news_page_urls = [news_tag.get_attribute('href') for news_tag in news_tags]

        for news_page_url in news_page_urls:
            driver.get(news_page_url)

            img_box_xpath = '/html/body/div[1]/div[3]/div[1]/div[6]/div[@class="img-box"]'
            img_box_tags = driver.find_elements(
                By.XPATH,
                img_box_xpath
            )

            if img_box_tags:
                for img_box_tag in img_box_tags:
                    wait = WebDriverWait(img_box_tag, 5)
                    try:
                        img_tag = wait.until(
                            EC.visibility_of_element_located(
                                (
                                    By.TAG_NAME,
                                    'img'
                                )
                            )
                        )

                        img_caption = wait.until(
                            EC.visibility_of_element_located(
                                (
                                    By.TAG_NAME,
                                    'em'
                                )
                            )
                        )
                    except:
                        continue

                    img_url = img_tag.get_attribute('src')
                    if img_url[-3:] == 'gif':
                        continue
                    img_caption = img_caption.text

                    if img_caption == '':
                        continue

                    img_url_resp = requests.get(img_url)
                    try:
                        img = Image.open(
                            BytesIO(img_url_resp.content)
                        )
                    except:
                        print(news_page_url)
                        continue

                    if img.mode == "P":
                        img = img.convert("RGB")

                    img_name = f'IMG_{img_idx:06}.jpg'
                    img_save_path = os.path.join(images_dir, img_name)
                    img.save(img_save_path)
                    img_idx += 1

                    caption_file_line_content = img_save_path + '\t' + img_caption +'\n'
                    caption_file.write(caption_file_line_content)

            driver.back()
except Exception as e:
    print(e)
    print(news_page_url)
    pass

caption_file.close()

<ipython-input-3-22bd616a5579>:4: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  chrome_options.headless = True
 88%|████████▊ | 87/99 [1:55:28<16:19, 81.61s/it]

https://kienthuckhoahoc.org/kh-vu-tru/tau-vu-tru-cho-nu-phi-hanh-gia-dau-tien-len-tram-thien-cung-bcjx


100%|██████████| 99/99 [2:10:10<00:00, 78.89s/it]


In [5]:
!zip -r space_image_captioning_dataset_100.zip space_image_captioning_dataset

  adding: space_image_captioning_dataset/ (stored 0%)
  adding: space_image_captioning_dataset/space_captions.txt (deflated 79%)
  adding: space_image_captioning_dataset/space_images/ (stored 0%)
  adding: space_image_captioning_dataset/space_images/IMG_000789.jpg (deflated 2%)
  adding: space_image_captioning_dataset/space_images/IMG_001129.jpg (deflated 2%)
  adding: space_image_captioning_dataset/space_images/IMG_000246.jpg (deflated 2%)
  adding: space_image_captioning_dataset/space_images/IMG_000525.jpg (deflated 1%)
  adding: space_image_captioning_dataset/space_images/IMG_000466.jpg (deflated 3%)
  adding: space_image_captioning_dataset/space_images/IMG_001836.jpg (deflated 8%)
  adding: space_image_captioning_dataset/space_images/IMG_000258.jpg (deflated 3%)
  adding: space_image_captioning_dataset/space_images/IMG_000136.jpg (deflated 1%)
  adding: space_image_captioning_dataset/space_images/IMG_001551.jpg (deflated 1%)
  adding: space_image_captioning_dataset/space_images/IMG

In [6]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [7]:
!cp '/content/space_image_captioning_dataset_100.zip' '/content/gdrive/MyDrive/Coordinate/aio_2022_ta/module_7/image_captioning/dataset'